#### skycoach data direect

In [4]:
# pip install playwright
# python -m playwright install chromium
import json, re, time
from pathlib import Path
from playwright.sync_api import sync_playwright

URL = "https://skycoach.gg/destiny-boost/products/opaque-hourglass-8613"

def sniff_product(url):
    blobs = []

    with sync_playwright() as p:
        browser = p.chromium.launch(headless=True, args=["--disable-blink-features=AutomationControlled"])
        ctx = browser.new_context(
            user_agent=("Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                        "AppleWebKit/537.36 (KHTML, like Gecko) "
                        "Chrome/124.0.0.0 Safari/537.36"),
            locale="en-US",
            timezone_id="Europe/London",
        )
        page = ctx.new_page()

        def on_response(resp):
            try:
                ct = resp.headers.get("content-type","")
                if "json" in ct:
                    j = resp.json()
                    # Keep likely product payloads (heuristic)
                    text = json.dumps(j).lower()
                    if any(k in text for k in ["variant","variants","options","sku","price"]):
                        blobs.append({"url": resp.url, "json": j})
            except Exception:
                pass

        page.on("response", on_response)
        page.goto(url, wait_until="networkidle", timeout=90_000)

        # Also try to read any JS globals rendered into DOM
        # Next.js / Nuxt / Redux states:
        for expr in ["window.__NEXT_DATA__", "window.__NUXT__", "window.__INITIAL_STATE__", "window.ShopifyAnalytics && window.ShopifyAnalytics.meta"]:
            try:
                obj = page.evaluate(expr)
                if obj:
                    blobs.append({"url": f"eval:{expr}", "json": obj})
            except Exception:
                pass

        # Save for inspection
        Path("captures").mkdir(exist_ok=True)
        for i, b in enumerate(blobs):
            Path("captures", f"payload_{i}.json").write_text(json.dumps(b, indent=2))

        browser.close()

    return blobs

def pick_best(blobs):
    best = None
    best_score = 0
    for b in blobs:
        text = json.dumps(b["json"]).lower()
        score = sum(text.count(k) for k in ["variants","variant","option1","option2","sku","price","product"])
        if score > best_score:
            best, best_score = b, score
    return best

if __name__ == "__main__":
    blobs = sniff_product(URL)
    if not blobs:
        raise RuntimeError("No JSON captured.")
    best = pick_best(blobs)
    print("Picked:", best["url"])
    # From here, map best["json"] into your schema.
    print(json.dumps(best["json"], indent=2)[:2000], "...\n\nSaved all in ./captures/")


Error: It looks like you are using Playwright Sync API inside the asyncio loop.
Please use the Async API instead.